In [27]:
import numpy as np
import pandas as pd 
from water_quality.missingprocessor import Processor
import pickle

data_path = "data"
loc = "DATA_FOR_GAN_WITH_DO_TRAIN"
seq_len = 365
df = pd.read_csv('{}/{}.csv'.format(data_path,loc), sep = ",", parse_dates=["Date"])
# Vérifier le type des colonnes
print("Types des colonnes avant traitement :\n", df.dtypes)

# Sauvegarder la colonne 'Date'
date_col = df["Date"]

# Exclure la colonne 'Date' car elle n'est pas numérique
numeric_data = df.drop(columns=["Date"])

print(numeric_data)

types = ["continuous" for i in range(len(numeric_data.columns))]

P = Processor(types)
print(types, P)
# Flip the data to make chronological data
ori_data = P.fit_transform(numeric_data)

print("Données originales :\n", numeric_data)
print("Données transformées :\n", ori_data)

# Réintégrer la colonne 'Date'
transformed_df = pd.DataFrame(ori_data, columns=numeric_data.columns)
#transformed_df["Date"] = date_col.values
transformed_df.insert(0, "Date", date_col.values)  # Place 'Date' en première position
print(transformed_df.columns)
print(transformed_df.head())


ori_data = ori_data[::-1]

temp_data = [ori_data[i:i + seq_len] for i in range(0, len(ori_data) - seq_len)]   

print(temp_data[:3])


Types des colonnes avant traitement :
 Date              datetime64[ns]
Protok_NS                  int64
Temperatura_NS           float64
DO_NS                    float64
Protok_Z                 float64
Temperatura_Z            float64
DO_Z                     float64
Protok_S                   int64
Temperatura_S            float64
DO_S                     float64
dtype: object
      Protok_NS  Temperatura_NS   DO_NS     Protok_Z  Temperatura_Z  \
0          2680            12.0   9.400  2725.773196      11.955788   
1          2550            12.5  10.200  2754.639175      12.455788   
2          2430            12.5   9.800  2595.876289      12.455788   
3          2320            12.5   9.700  2494.845361      12.455788   
4          2230            12.5   9.200  2480.412371      12.455788   
...         ...             ...     ...          ...            ...   
2987       1600             5.6  12.025  2105.154639       5.300000   
2988       1720             5.6  12.200  2292.783

In [15]:
!pip install fastNLP


  Using cached FastNLP-1.0.1-py3-none-any.whl
  Using cached prettytable-3.12.0-py3-none-any.whl (31 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
  Using cached rich-11.2.0-py3-none-any.whl (217 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


In [16]:

from fastNLP import DataSet
dataset = DataSet({"seq_len": [seq_len] * len(temp_data), "dyn": temp_data, "sta":[0]*len(temp_data)})
dic = {
    "train_set": dataset,
    "dynamic_processor": P,
    "static_processor": Processor([])
}
print(P.dim, len(temp_data))
from utils.general import make_sure_path_exists
make_sure_path_exists("./data")
with open("./data/{}.pkl".format(loc), "wb") as f:
    pickle.dump(dic, f)

9 2627
